<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_11/blob/main/multidocQA_using_LLM_and_IIRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import tqdm

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"

    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/aula_11"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    # Setup Anserini executables

    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
    os.environ["ANSERINI_CLASSPATH"]="/content/drive/MyDrive/unicamp/ia368v_dd/anserini/target"

    !chmod +x /content/drive/MyDrive/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval
    

    !pip install pyserini -q
    !pip install faiss-cpu -q

    PYSERINI_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/pyserini/"

    TREC_EVAL_FULLPATH=PYSERINI_FOLDER+"tools/eval/trec_eval.9.0.4/trec_eval"
    
    PYTHON="python3"
    
    !pip install transformers -q

    !pip install openai -q

    #
    # Install pygaggle
    #
    
    !git clone --recursive https://github.com/castorini/pygaggle.git
    !pip install -q -r pygaggle/requirements.txt
    !cp -r pygaggle/pygaggle /usr/local/lib/python3.10/dist-packages
    
else:
    PYTHON="python3.8"
    
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_11/"
    PYSERINI_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/"
    
    TREC_EVAL_FULLPATH=PYSERINI_FOLDER+"tools/eval/trec_eval.9.0.4/trec_eval"
    
    os.environ["ANSERINI_CLASSPATH"]="/media/eduseiti/bigdata01/unicamp/ia368v_dd/anserini/target"
    
PYSERINI_TOOLS_FOLDER=PYSERINI_FOLDER+"tools/scripts/msmarco/"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 118.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/

In [4]:
import json
import pickle
import numpy as np
import pandas as pd

from pyserini.search.lucene import LuceneSearcher

import spacy

import re

import time

from pyserini.search import LuceneSearcher

In [5]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

In [6]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

import torch

In [22]:
import openai

In [7]:
import collections
import string
import unicodedata

In [57]:
WIKIPEDIA_CONTEXT_ARTICLES="context_articles.json"
DEV_QUESTIONS="dev.json"
TRAIN_QUESTIONS="train.json"
TEST_QUESTIONS="iirc_test.json"

In [9]:
IIRC_BM25_FOLDER="iirc_processed"
IIRC_INDEX_FOLDER=IIRC_BM25_FOLDER + "/index"

In [10]:
MAX_SENTENCES_PER_DOCUMENT=3
SENTENCES_STRIDE=2

In [11]:
API_KEYS_FILE="../api_keys_20230324.json"

### Define a function to create the BM25 index

In [12]:
def create_BM25_index(corpus_json_file,
                      questions_files_list,
                      converted_docs_folder,
                      index_folder, 
                      max_sentences=MAX_SENTENCES_PER_DOCUMENT, 
                      sentences_stride=SENTENCES_STRIDE):
    
    
    start_time = time.time()
    
    #
    # Check if the index already exists
    #
    
    if os.path.exists(index_folder):
        print("BM25 index already created...")
        
        pyserini_format_docs = []
        
        with open(os.path.join(converted_docs_folder, "contents.jsonl"), "r") as input_file:
            for line in input_file:
                pyserini_format_docs.append(json.loads(line))       
    else:
        if not os.path.exists(os.path.join(converted_docs_folder, "contents.jsonl")):
            with open(corpus_json_file) as input_file:
                corpus_json = json.load(input_file)
                
            print("Original context corpus number of documents: {}".format(len(corpus_json)))
            

            #
            # Append the documents inside the questions files
            #

            for questions_file in questions_files_list:
                with open(questions_file) as input_file:
                    questions = json.load(input_file)
                
                print("\nAdding {} documents from {}...".format(len(questions), questions_file))
                
                for question in questions:
                    if not question['title'].lower() in corpus_json:
                        corpus_json[question['title'].lower()] = question['text']
                    else:
                        print("- \"{}\" already in the original corpus.".format(question['title'].lower()))

                        
            print("\nTotal number of documents: {}".format(len(corpus_json)))
            

            #
            # Computing the windowed version of the corpus document was based no the
            # following implementation: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb
            #

            nlp = spacy.blank("en")
            nlp.add_pipe("sentencizer")

            pyserini_format_docs = []

            for i, (document_title, document_original_text) in enumerate(tqdm.tqdm(corpus_json.items(), bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}', colour='GREEN', file=sys.stdout)):
                cleaned_text = re.sub('<[^<]+?>', '', document_original_text)

                # print(cleaned_text)

                processed_text = nlp(cleaned_text)

                # print([type(sentence) for sentence in processed_text.sents])

                text_sentences = [sentence.text.strip() for sentence in processed_text.sents]

                for j in range(0, len(text_sentences), sentences_stride):
                    document_part = ' '.join(text_sentences[j:(j + max_sentences)])

                    pyserini_format_docs.append({'title': document_title,
                                                'contents': document_title + ". " + document_part,
                                                'document_part': document_part})

                    if j + max_sentences >= len(text_sentences):
                        break

            print("len(pyserini_format_docs)={}".format(len(pyserini_format_docs)))

            if not os.path.exists(converted_docs_folder):
                os.makedirs(converted_docs_folder)

            with open(os.path.join(converted_docs_folder, "contents.jsonl"), "w") as output_file:
                for i, document in enumerate(pyserini_format_docs):

                    # Filter-off empty documents ― could have been generated during the sentencizer step...

                    if document['document_part'] != "":

                        if i > 0:
                            output_file.write("\n")

                        document['id'] = i
                        output_file.write(json.dumps(document))
        else:
            print("Have already created the contents.jsonl file...")

        #
        # Now creates the BM25 index
        #

        current_folder = os.getcwd()

        os.chdir(PYSERINI_FOLDER)

        !{PYTHON} -m pyserini.index.lucene \
            --collection JsonCollection \
            --input {current_folder}/{converted_docs_folder} \
            --index {current_folder}/{index_folder} \
            --generator DefaultLuceneDocumentGenerator \
            --threads 9 \
            --storePositions --storeDocvectors --storeRaw

        os.chdir(current_folder)
    
    return pyserini_format_docs, time.time() - start_time

### Define functions to process the answers and compute the metrics

These functions are the exact copy from https://github.com/neuralmind-ai/visconde/blob/main/iirc_generate_and_evaluate.ipynb

In [13]:
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  def remove_accents(input_str):
      nfkd_form = unicodedata.normalize('NFKD', input_str)
      only_ascii = nfkd_form.encode('ASCII', 'ignore')
      return only_ascii.decode("utf-8")

  return white_space_fix(remove_articles(remove_punc(lower(remove_accents(s)))))


In [14]:
def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

In [15]:
def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

In [16]:
def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

### Define functions to execute the documents search and the LLM query

In [39]:
def execute_documents_search(which_query, BM25_searcher, reranker, k=1000):

    search_start_time = time.time()

    document_hits = BM25_searcher.search(which_query, k=k)

    #
    # Document reranking; implementation based on https://github.com/neuralmind-ai/visconde/blob/main/iirc_rerank.ipynb
    #

    reranker_texts = []

    for document in document_hits:
        document_dict = json.loads(document.raw)
        reranker_texts.append(Text(document_dict['contents'], {'docid': document_dict['id'],
                                                               'title': document_dict['title'],
                                                               'text': document_dict['contents']}))

    reranker_query = Query(which_query)
        
    reranking_start = time.time()
    reranked_documents = reranker.rerank(reranker_query, reranker_texts)
    reranking_end = time.time()

    reranked_documents.sort(key=lambda x: x.score, reverse=True)

    final_documents_list = []

    for reranked_doc in reranked_documents:
        doc_data = reranked_doc.metadata
        doc_data['score'] = reranked_doc.score

        final_documents_list.append(doc_data)

    final_time = time.time()

    print("Search and reranking duration: {}; reranking duration: {}\n".format(final_time - search_start_time, reranking_end - reranking_start))

    return final_documents_list, final_time - search_start_time

In [45]:
def execute_LLM_document_aggregation(which_query, reranked_documents, openai, k=3, verbose=True):

    start_time = time.time()

    # Starting to build the messages to send adding the prompt and few shot examples

    messages_to_send = [SYSTEM_ROLE]
    messages_to_send += [example_role for example in EXAMPLES for example_role in example]

    # Now add the supporting passages

    question=EXAMPLE_FORMAT.format(len(EXAMPLES) + 1)

    for i, doc in enumerate(reranked_documents[:k]):
        question += DOCUMENT_FORMAT.format(i + 1, doc['title'], doc['text'])

    question += QUESTION_FORMAT.format(which_query)

    if verbose:
        print(question)

    messages_to_send.append({'role': "user", 'content': question})

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                            messages=messages_to_send,
                                            temperature=0,
                                            max_tokens=1000)
    if verbose:
        print("\n")
        print(response['choices'][0]['message']['content'])

    m = re.match(OPENAI_RESPONSE_REGEX, response['choices'][0]['message']['content'])

    if len(m.groups()) == 2:
        evidence = m.group(1)
        answer = m.group(2)
    else:
        evidence = None
        answer = None

    final_time = time.time()

    print("\nLLM document aggregation duration: {}".format(final_time - start_time))

    return {'answer': answer,
            'evidence': evidence}, final_time - start_time

In [29]:
def return_final_answer(which_iirc_question):
    
    #
    # Implementation based on https://github.com/neuralmind-ai/visconde/blob/main/iirc_generate_and_evaluate.ipynb
    #
    
    final_answer = []
    
    if which_iirc_question['answer']['type'] == "span":
        final_answer = ", ".join([answer_span['text'] for answer_span in which_iirc_question['answer']["answer_spans"]])

    elif which_iirc_question['answer']['type'] == "value":
        final_answer = "{0} {1}".format(which_iirc_question['answer']['answer_value'], 
                                        which_iirc_question['answer']['answer_unit'])
        
    elif which_iirc_question['answer']['type'] == "binary":
        final_answer = which_iirc_question['answer']['answer_value']
        
    elif which_iirc_question['answer']['type'] == "none":
        final_answer = "Not enough information"

    return {'question': which_iirc_question['question'],
            'final_answer': final_answer}

In [65]:
def execute_queries_from_test_set(test_set_filename, BM25_params, LLM_params, number_of_docs=NUMBER_OF_TEST_DOCS, verbose=True):

    with open(test_set_filename) as input_file:
        test_set = json.load(input_file)

    selected_documents = np.random.choice(list(range(len(test_set))), number_of_docs, replace=False)

    selected_questions=[]

    for document in selected_documents:
        doc_questions = test_set[document]['questions']
        
        selected_question = int(np.random.choice(list(range(len(doc_questions))), 1))
        
        if verbose:
            print("Selected question {} for document {}".format(selected_question, document))
        
        final_answer = return_final_answer(doc_questions[selected_question])
        
        final_answer['doc_id'] = document
        final_answer['question_id'] = selected_question

        selected_questions.append(final_answer)

    for question in selected_questions:

        reranked_documents, search_duration = execute_documents_search(question['question'], **BM25_params)
        computed_answer, LLM_duration = execute_LLM_document_aggregation(question['question'], reranked_documents, **LLM_params, verbose=verbose)

        if computed_answer['answer'] is not None:
            exact_match_score = compute_exact(question['final_answer'], computed_answer['answer'])
            f1_score = compute_f1(question['final_answer'], computed_answer['answer'])
        else:
            exact_match_score = None
            f1_score = None

        question['computed_answer'] = computed_answer['answer']
        question['computed_evidence'] = computed_answer['evidence']
        question['exact_match_score'] = exact_match_score
        question['f1_score'] = f1_score
        question['duration'] = search_duration + LLM_duration

    return pd.DataFrame(selected_questions)

### Define prompt for the LLM request

In [17]:
SYSTEM_ROLE = {
    'role': "system", 
    'content': "You are an assistant who will read some documents and answer to a question considering only the information available in those documents, providing evidence to support your answer, following the example provided."
}

#### The examples below were taken from https://github.com/neuralmind-ai/visconde/blob/main/iirc_generate_and_evaluate.ipynb

In [18]:
EXAMPLES=[
    [
        {
            'role': "user",
            'content': "Example 1:\n\nDocument 1: Title: Fred J. Shields. Content: Ollie Murphy's first-half goal gave 'the Royals' a huge boost at half-time.\n\nDocument 2: Title: Ollie Murphy. Content: He plays club football for Carnaross\n\nDocument 3: Title: Ollie Murphy. Content: He came to national prominence in 1999 when he was one of Meath's best player's\n\nQuestion: Based on the above documents, Did Ollie Murphy play for any teams other than 'the Royals'?", 
        },
        {  
            'role': "assistant",
            'content': "Evidence: Document 1 says that Ollie Murphy's first-half goal gave 'the Royals' a huge boost at half-time. However, this does not necessarily mean that Ollie only played for 'the Royals'. Document 2 states that Ollie plays club football for Carnaross. This suggests that Ollie may have played for other teams in addition to 'the Royals'. Document 3 says that Ollie came to national prominence in 1999 when he was one of Meath's best players. This also suggests that Ollie has played for other teams. Therefore, it is likely that Ollie has played for teams other than 'the Royals'.\n\nAnswer: yes."
        }
    ],

    [
        {
            'role': "user",
            'content': "Example 2:\n\nDocument 1: Title: Don Rendell. Content: The club played in the Brunei Premier League in the early 2000s, winning the league title in 2002 and 2004.\n\nQuestion: Based on the above documents, What club came in second at the 2004 Brunei Premier League?",
        },
        {
            'role': "assistant",
            'content': "Evidence: There is not enough information in the documents to answer the question.\n\nAnswer: not enough information."
        }
    ],

    [
        {
            'role': "user",
            'content': "Example 3:\n\nDocument 1: Title: Stacy Compton. Content: Despicable Me, the first film in the series, and the first film from Illumination, was released on July 9, 2010.\n\nDocument 2: Title: Miranda Cosgrove. Content: Cosgrove's first television appearance (aside from commercials) was in 2001 as the voice of young Lana Lang in the pilot episode of Smallville.\n\nDocument 3: Title: Miranda Cosgrove. Content: In 2004, Cosgrove soon landed her first major role in a television series when she was awarded a main role in the Nickelodeon series Drake & Josh.\n\nDocument 4: Title: Miranda Cosgrove. Content: Also in 2004, Cosgrove guest-starred in a special episode of the animated series What\'s New, Scooby-Doo?, as well as guest-starring in a season five episode of Grounded For Life.\n\nDocument 5: Title: Miranda Cosgrove. Content: The television series, which aired on Disney, is a spin-off of the original film, Lilo & Stitch.\n\nDocument 6: Title: Miranda Cosgrove. Content: The first of these appearances was in Zoey 101. Cosgrove later guest starred on an episode of Unfabulous,\n\nDocument 7: Title: Miranda Cosgrove. Content: However, Cosgrove was already in the works of starring in her own sitcom, titled iCarly, released on September 8, 2007.\n\nQuestion: Based on the above documents, How many TV shows had Miranda Cosgrove been featured in by the year Despicable Me was released?",
        },
        {
            'role': "assistant",
            'content': "Evidence: According to document 1, Despicable Me was released on July 9, 2010.Document 2 states that Cosgrove's first television appearance was in 2001 as the voice of young Lana Lang in the pilot episode of Smallville. Document 3 says that, in 2004, Cosgrove landed her first major role in a television series when she was awarded a main role in the Nickelodeon series Drake & Josh.Document 4 states that, also in 2004, Cosgrove guest-starred in a special episode of the animated series What\'s New, Scooby-Doo?, as well as guest-starring in a season five episode of Grounded For Life.Document 5 says that the television series, which aired on Disney, is a spin-off of the original film, Lilo & Stitch.Document 6 states that the first of these appearances was in Zoey 101. Cosgrove later guest starred on an episode of Unfabulous. Document 7 says that, however, Cosgrove was already in the works of starring in her own sitcom, titled iCarly, released on September 8, 2007. Therefore, Miranda Cosgrove had been featured in 8 TV shows by the year Despicable Me was released.\n\nAnswer: 8 TV shows."
        }
    ]
]

In [19]:
EXAMPLE_FORMAT="Example {}:\n\n"
DOCUMENT_FORMAT="Document {}: Title: {} Content: {}\n\n"
QUESTION_FORMAT="Question: {}"

OPENAI_RESPONSE_REGEX="[\n\r]*[Ee]vidence:\s+(.+)[\n\r]*[Aa]nswer:\s+(.+)[\n\r]*"

In [51]:
NUMBER_OF_TEST_DOCS = 10
MAX_QUERIES_PER_DOC = 1

### Declare supporting elements

In [23]:
with open(API_KEYS_FILE) as inputFile:
    api_keys = json.load(inputFile)

openai.api_key = api_keys['OPENAI_API_KEY']

In [24]:
BM25_searcher = LuceneSearcher(IIRC_INDEX_FOLDER)

In [25]:
model = T5ForConditionalGeneration.from_pretrained("castorini/monot5-3b-msmarco-10k", torch_dtype=torch.float16)

device = torch.device("cuda")
model.to(device)

reranker = MonoT5(model=model)

For stability purposes, it is recommended to have accelerate installed when using this model in torch.float16, please install it with `pip install accelerate`


### Create the BM25 index

In [26]:
all_documents, creation_time = create_BM25_index(WIKIPEDIA_CONTEXT_ARTICLES, 
                                                 ["dev.json", "train.json", "iirc_test.json"], 
                                                 IIRC_BM25_FOLDER, 
                                                 IIRC_INDEX_FOLDER)

BM25 index already created...


### Execute the queries

In [48]:
BM25_PARAMETERS={
    "BM25_searcher": BM25_searcher, 
    "reranker": reranker, 
    "k": 1000
}

LLM_PARAMETERS={
   "openai": openai,
   "k": 3
}

In [52]:
result = execute_queries_from_test_set(DEV_QUESTIONS, BM25_PARAMETERS, LLM_PARAMETERS, verbose=True)

Selected question 2 for document 12
Selected question 1 for document 37
Selected question 2 for document 159
Selected question 0 for document 152
Selected question 1 for document 271
Selected question 1 for document 289
Selected question 0 for document 384
Selected question 4 for document 288
Selected question 0 for document 306
Selected question 0 for document 100
Search and reranking duration: 13.384358882904053; reranking duration: 12.910121202468872

Example 4:

Document 1: Title: die gezeichneten Content: die gezeichneten. It was staged, albeit heavily cut, at the Salzburg Festival in 2005, directed by Nikolaus Lehnhoff and conducted by Kent Nagano, with Robert Brubaker (Alviano) and Anne Schwanewilms (Carlotta) in the leading roles. The American premiere was staged by Los Angeles Opera on 10 April 2010, followed by three more performances. James Conlon conducted.

Document 2: Title: die gezeichneten Content: die gezeichneten. Conductor Michael Gielen revived the opera at the Oper

In [53]:
result

,question,final_answer,doc_id,question_id,computed_answer,computed_evidence,exact_match_score,f1_score,duration
0,How old was Kent Nagano the year that he condu...,54 years,12,2,53 years old.,"According to document 1, Kent Nagano conducted...",0,0.400000,21.258298
1,In what city did the team that Cantillo signed...,Not enough information,37,1,not enough information.,Document 1 states that Cantillo signed with th...,1,1.000000,26.444447
2,Had Walt Disney World Speedway been standing l...,Nazareth Speedway,159,2,not enough information.,Document 1 states that Walt Disney World Speed...,0,0.000000,23.531391
3,What age was Henry Willis when he made the two...,48 years,152,0,not enough information.,Document 1 and 2 both mention that the two-man...,0,0.000000,23.167412
4,What player was drafted second in the 1975 MLB...,Not enough information,271,1,not enough information.,There is no information in the documents about...,1,1.000000,15.901799
5,When did the player Vesnina lost to in three s...,Not enough information,289,1,not enough information.,There is no mention of the Rogers Cup in any o...,1,1.000000,18.848022
6,How many denizens lived in Paris the year Corr...,Not enough information,384,0,not enough information.,There is no information in the documents about...,1,1.000000,16.063816
7,How many students did the University of Michig...,Not enough information,288,4,not enough information.,There is no information in the documents about...,1,1.000000,16.685015
8,Was the Palm IIIe or Palm IIIx released first?,The Palm IIIx,306,0,The Palm IIIx was released first.,"According to document 1, the Palm IIIx was rel...",0,0.571429,20.681486
9,Which of the series Square Enix is famous for ...,Not enough information,100,0,Final Fantasy.,"According to document 1, the Dragon Quest fran...",0,0.000000,22.334173


In [55]:
result[['exact_match_score', 'f1_score', 'duration']].mean()

exact_match_score     0.500000
f1_score              0.597143
duration             20.491586
dtype: float64

In [58]:
result = execute_queries_from_test_set(TEST_QUESTIONS, BM25_PARAMETERS, LLM_PARAMETERS, verbose=True)

Selected question 2 for document 501
Selected question 0 for document 119
Selected question 0 for document 300
Selected question 0 for document 483
Selected question 1 for document 69
Selected question 0 for document 89
Selected question 0 for document 319
Selected question 2 for document 336
Selected question 1 for document 424
Selected question 1 for document 148
Search and reranking duration: 13.075116872787476; reranking duration: 12.975397825241089

Example 4:

Document 1: Title: hotchkiss h35 Content: hotchkiss h35. No less troublesome was the gearbox: it was difficult to engage the highest fifth gear and so the theoretical top speed of 27.8 km/h was rarely reached. There was one reverse gear. The inevitable rough handling of the tank by the driver resulted in much wear and tear.

Document 2: Title: hotchkiss h35 Content: hotchkiss h35. And so the first 300 vehicles of the production run were offered to the Cavalry, which was forced to accept them because it would not have been g

In [59]:
result

,question,final_answer,doc_id,question_id,computed_answer,computed_evidence,exact_match_score,f1_score,duration
0,What is the top speed of the three French tank...,28 km/h,501,2,not enough information.,Document 3 mentions that three Hotchkiss tanks...,0,0.000000,20.964653
1,In which country did Rk Torfs contributed as a...,Belgium,119,0,not enough information.,"According to document 1, Rik Torfs became a we...",0,0.000000,18.254638
2,Which town that borders Wolf trap has the high...,Reston,300,0,not enough information.,There is no information in the documents about...,0,0.000000,16.720152
3,Is the midfielder who scored an equaliser agai...,yes,483,0,not enough information.,"According to document 2, FK Kukësi played agai...",0,0.000000,21.176766
4,What is the difference in population between t...,2969881 people,69,1,not enough information.,There is no information in the documents about...,0,0.000000,17.334922
5,How old was Mario Morales when he won the cham...,32 years,89,0,not enough information.,There is no information in the documents about...,0,0.000000,15.074987
6,How old was Marie Battiste when she brought he...,29 years,319,0,Around 29 years old.,There is no information in the documents that ...,0,0.666667,24.257460
7,Who were judges on the Romanian Supreme Court ...,Not enough information,336,2,not enough information.,There is no information in the documents about...,1,1.000000,18.879478
8,What is the student enrollment of the school w...,"6,000",424,1,not enough information.,"According to documents 1 and 2, Barbara Kolb r...",0,0.000000,21.982890
9,For how many years was Corinth goverened by th...,185 years,148,1,not enough information.,"According to document 2, Corinth became a full...",0,0.000000,19.831744


In [60]:
result[['exact_match_score', 'f1_score', 'duration']].mean()

exact_match_score     0.100000
f1_score              0.166667
duration             19.447769
dtype: float64

In [61]:
LLM_PARAMETERS_k5={
   "openai": openai,
   "k": 5
}

In [62]:
result = execute_queries_from_test_set(TEST_QUESTIONS, BM25_PARAMETERS, LLM_PARAMETERS_k5, verbose=True)

Selected question 2 for document 342
Selected question 0 for document 276
Selected question 1 for document 222
Selected question 0 for document 388
Selected question 2 for document 258
Selected question 1 for document 16
Selected question 2 for document 287
Selected question 1 for document 282
Selected question 1 for document 361
Selected question 1 for document 71
Search and reranking duration: 13.577125549316406; reranking duration: 13.476476907730103

Example 4:

Document 1: Title: thomas c. platt Content: thomas c. platt. Thomas Collier Platt (July 15, 1833 – March 6, 1910) was a two-term member of the U.S. House of Representatives (1873–1877) and a three-term U.S. Senator from New York in 1881 and 1897 to 1909. He is best known as the "political boss" of the Republican Party in New York State in the late 19th century and early 20th century. Upon his death, the New York Times stated that "no man ever exercised less influence in the Senate or the House of Representatives than he," b

In [63]:
result

,question,final_answer,doc_id,question_id,computed_answer,computed_evidence,exact_match_score,f1_score,duration
0,How old was Thomas C. Platt when Barnes began ...,78 years,342,2,not enough information.,There is no information in the documents about...,0,0.000000,18.728590
1,How old was King Louis XVI of France when he s...,38 years,276,0,37 years old.,"According to document 2, a formal alliance was...",0,0.400000,23.599910
2,Which talk show had a longer run on television...,Rivera Live,222,1,Rivera Live.,Document 1 mentions that Geraldo Rivera ended ...,1,1.000000,22.463043
3,During what years were hill forts developed?,entire 2nd millennium BC,388,0,Hill forts were developed roughly during the s...,Document 2 states that hill forts developed in...,0,0.266667,19.380731
4,What is the birthdate of the writer who descri...,"May 19, 1789",258,2,not enough information.,"According to document 1, Eugene W. Hilgard was...",0,0.000000,29.206773
5,What was the population of Cape Town when Hari...,Not enough information,16,1,not enough information.,There is no information in the documents about...,1,1.000000,18.357666
6,How old was Gardiner Greene Hubbard when he bu...,66 years,287,2,66 years old.,"According to document 1, Gardiner Greene Hubba...",0,0.800000,22.472733
7,How many years between Haysbert's appearance o...,6 years,282,1,not enough information.,"According to document 2, Haysbert played Detec...",0,0.000000,20.342650
8,How many men total was General Gracie in comma...,Not enough information,361,1,not enough information.,Document 1 and Document 2 both mention that Ge...,1,1.000000,25.417608
9,How old was Lucille Ball when her show on CBS ...,Not enough information,71,1,not enough information.,"According to document 1, Lucille Ball starred ...",1,1.000000,19.225885


In [64]:
result[['exact_match_score', 'f1_score', 'duration']].mean()

exact_match_score     0.400000
f1_score              0.546667
duration             21.919559
dtype: float64

In [66]:
result = execute_queries_from_test_set(TEST_QUESTIONS, BM25_PARAMETERS, LLM_PARAMETERS_k5, number_of_docs=30, verbose=True)

Selected question 1 for document 112
Selected question 0 for document 465
Selected question 0 for document 378
Selected question 0 for document 362
Selected question 5 for document 470
Selected question 1 for document 152
Selected question 0 for document 168
Selected question 0 for document 108
Selected question 1 for document 131
Selected question 3 for document 190
Selected question 0 for document 49
Selected question 0 for document 438
Selected question 0 for document 319
Selected question 0 for document 469
Selected question 0 for document 58
Selected question 2 for document 480
Selected question 0 for document 93
Selected question 3 for document 126
Selected question 0 for document 459
Selected question 2 for document 490
Selected question 0 for document 6
Selected question 0 for document 488
Selected question 2 for document 381
Selected question 3 for document 442
Selected question 3 for document 57
Selected question 0 for document 231
Selected question 0 for document 177
Selecte

In [67]:
result

,question,final_answer,doc_id,question_id,computed_answer,computed_evidence,exact_match_score,f1_score,duration
0,"Which period last longer, the Tang Dynasty or ...",The Tang dynasty,112,1,The Tang Dynasty lasted longer than the Five D...,Document 1 states that the Five Dynasties and ...,0,0.307692,24.381076
1,How long did the assault on Guadalcanal last?,6 months,465,0,Approximately 6 months.,"According to document 1, the Guadalcanal campa...",0,0.800000,36.174942
2,What was Bloemfontein before it was a Free State?,a fort,378,0,Orange River Sovereignty.,"According to document 1, Bloemfontein was the ...",0,0.000000,26.839804
3,What countries have coastlines at the sea wher...,"Spain, France, Monaco, Italy, Slovenia, Croati...",362,0,"Spain, France, Monaco, Italy, Slovenia, Croati...",Document 1 states that the sea beet is native ...,0,0.717949,32.678094
4,Has the team Stockdale signed with in 2008 or ...,Not enough information,470,5,not enough information.,"According to document 1, David Stockdale signe...",1,1.000000,25.059552
5,Which Pay-Per-View event that El Torero partic...,Not enough information,152,1,not enough information.,There is no information in the documents about...,1,1.000000,16.348953
6,How long did the Civil War sparked by the atte...,9 years,168,0,three days.,Document 1 states that the Rochester race riot...,0,0.000000,19.553311
7,How old was Dontrelle Willis when he became 20...,21 years,108,0,not enough information.,"According to document 5, Willis was named the ...",0,0.000000,19.778715
8,Who was older during Alan Smith's ordination a...,Not enough information,131,1,It is not possible to determine who was older ...,"According to document 1, Alan Smith was ordain...",0,0.097561,46.992552
9,How much money did the show The Islanders make...,Not enough information,190,3,not enough information.,There is no information in the documents about...,1,1.000000,16.992897


In [68]:
result[['exact_match_score', 'f1_score', 'duration']].mean()

exact_match_score     0.266667
f1_score              0.353930
duration             25.481752
dtype: float64